# Test the linking of Compustat and CRSP

For documentation:
* [CRSP/Compustat Merged Database](http://www.crsp.com/products/research-products/crspcompustat-merged-database)
* [CRSP/Compustat Merged Database Guide Web](http://www.crsp.com/products/documentation/crspcompustat-merged-database-guide)
* [CRSP/Compustat Merged Database Guide PDF](http://www.crsp.com/files/ccm_data_guide_0.pdf)
* [From WRDS](https://wrds-www.wharton.upenn.edu/pages/support/applications/linking-databases/linking-crsp-and-compustat/)

#### Import Data

In [1]:
# import libraries
import pandas
import numpy as np
import datetime

In [2]:
# Import csv data
compustat = pandas.read_csv("Compustat19612018.csv")
# Used dtypes here to speed up function
crsp = pandas.read_csv("CRSP1960_2018.csv", \
    dtype={'PERMNO': int, 'date': str, 'EXCHCD': float, 'SICCD': str, 'SHRCLS': str, 'PRIMEXCH': str, \
           'PRC': float, 'RET': str, 'SHROUT': float, 'CFACSHR': float, 'vwretd': float, 'sprtrn': float})
cpi = pandas.read_csv("CPIData.csv")
linking_table = pandas.read_csv('CompustatMergedDatabase.csv')

### Link CRSP and Compustat

In [3]:
# Fix LINKDT and LINKENDDT dates
linking_table.LINKDT = pandas.to_datetime(linking_table.LINKDT.astype(str), format='%Y%m%d', errors='coerce').fillna(datetime.date.today())
linking_table.LINKENDDT = pandas.to_datetime(linking_table.LINKENDDT.astype(str), format='%Y%m%d', errors='coerce').fillna(datetime.date.today())

In [4]:
# Merge compustat and linking_table based on gvkey to get permnos for each compustat company
compustat_link = compustat.merge(linking_table, on='gvkey')

In [5]:
# Drop dates that are not in the effective range in linking_table
compustat_link.datadate = pandas.to_datetime(compustat_link.datadate.astype(str), format='%m/%d/%Y')
compustat_link = compustat_link[(compustat_link.datadate >= compustat_link.LINKDT) & (compustat_link.datadate <= compustat_link.LINKENDDT)]

In [6]:
# Drop data that will cause duplication
compustat_link = compustat_link[(compustat_link.indfmt == 'INDL') & (compustat_link.consol == 'C') & (compustat_link.popsrc == 'D') & (compustat_link.datafmt == 'STD')]

In [7]:
# Create year column in crsp to link with permno
crsp['year'] = pandas.to_datetime(crsp.date, format='%m/%d/%Y').dt.year;

In [8]:
# Create permnoyear column for crsp and compustat
crsp['permnoyear'] = crsp.PERMNO.map(str) + crsp.year.map(str)
compustat_link['permnoyear'] = compustat_link.LPERMNO.map(str) + compustat_link.fyear.map(str)

In [9]:
# Drop the permnoyear that are duplicated due to companies' mergings
compustat_link = compustat_link.sort_values(by=['permnoyear','datadate']).drop_duplicates(subset=['permnoyear'],keep='last')

In [10]:
# For each year, we need the most recent value, so I am goruping it by permnoyear and then
# just taking the tail (last member) of the group. Its quick and dirty and we probably want 
# a better way to do this so we know we are getting the last value if the data isn't sorted
crspGetLastYear = crsp[['permnoyear','EXCHCD','SICCD','SHROUT' ,'SHRCLS','PRIMEXCH', 'date']]
crspGetLastYear = crspGetLastYear.groupby('permnoyear').tail(1)

In [11]:
# These columns need to be summed for each year, which is being done here
# RET is a str column and sometimes has error characters so still need to handle that
crspSum = crsp[['permnoyear', 'PRC', 'RET', 'vwretd', 'sprtrn']]
crspSum = crspSum.groupby(by=['permnoyear'])['sprtrn', 'PRC', 'RET', 'vwretd', 'sprtrn'].sum()

In [12]:
# Merge the two annualized subset dataframes back together
crsp_link = crspSum.merge(crspGetLastYear, on='permnoyear')

In [13]:
# Merge crsp and compustat by permnoyear and convert permnoyear to ints
model_data = compustat_link.merge(crsp_link, on='permnoyear')
model_data["permnoyear"] = pandas.to_numeric(model_data["permnoyear"])

In [14]:
# Sort the merge table by permnoyear
model_data = model_data.sort_values(by=["permnoyear"])

In [15]:
print(model_data.columns)

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'curcd', 'act', 'at', 'capx', 'ceq', 'che', 'cogs', 'csho',
       'dlc', 'dltt', 'dp', 'ebit', 'ib', 'lct', 'lt', 'mib', 'mibt', 'pi',
       'pstk', 'pstkl', 'pstkrv', 're', 'revt', 'sale', 'seq', 'tie', 'txp',
       'txt', 'costat', 'conm', 'LINKPRIM', 'LIID', 'LINKTYPE', 'LPERMNO',
       'LPERMCO', 'LINKDT', 'LINKENDDT', 'permnoyear', 'sprtrn', 'PRC',
       'vwretd', 'sprtrn', 'EXCHCD', 'SICCD', 'SHROUT', 'SHRCLS', 'PRIMEXCH',
       'date'],
      dtype='object')


### Validation

In [16]:
print('Shape of model_data {}'.format(model_data.shape))
print('Shape of crsp {}'.format(crsp.shape))
print('Shape of compustat {}'.format(compustat.shape))

Shape of model_data (286967, 56)
Shape of crsp (4118557, 14)
Shape of compustat (474879, 37)


In [17]:
# Check if permnoyear is unique
compustat_link.groupby(['permnoyear']).size().sort_values(ascending=False).head(1)

permnoyear
934362017    1
dtype: int64